In this notebook, we use pyspark to explore a [dataset](https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data) from the San Francisco Police Department

In [5]:
!pip install pyspark

     |████████████████████████████████| 215.6MB 39kB/s eta 0:00:011    |█████████████████▊              | 119.2MB 30kB/s eta 0:51:53     |█████████████████████████▏      | 169.4MB 31kB/s eta 0:24:29
     |████████████████████████████████| 204kB 11kB/s eta 0:00:011
  Running setup.py install for pyspark ... done


In [16]:
!pip install ggplot

     |████████████████████████████████| 2.2MB 35kB/s eta 0:00:0199


In [10]:
import pyspark.sql

In [18]:
from csv import reader
from pyspark.sql import Row
from pyspark.sql.types import *

In [19]:
# read the data
crime_data_lines = sc.textFile('sf_data.csv')
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])

NameError: name 'sc' is not defined

In [ ]:
# extract header
header = df_crimes.first()
dispaly(header)

In [ ]:
# get anathing but the header
crimes = df_crimes.filter(lambda x: x!=header)
crimes.take(2) # check two rows

Now we can transform the RDD to Data fram, which is much easier to manipulate. At first, we need to convert RDD elements to RDD Row objects, we can define a function below to take a list of column names 9header) and a list of values (one row of our data) and then create a RDD Row of key-value pairs.

In [ ]:
def list_to_row(keys, values):
    row_dict = dict(zip(keys, values))
    return Row(**row_dict) # use *8 to unpack the dictionary to become "key = value" pairs

We can now use map and the function defined above to convert all the rows of our data to RDD Row objects, and then create data frame using ``spark.createDataFrame``.

In [ ]:
rdd_rows = crimes.map(lambda x: list_to_row(header, x))
df = spark.createDataFrame(rdd_rows)

In [ ]:
# take a look at the data frame
display(df)

Replace two column names, 'X' and 'Y', with 'Longitude' and 'Latitude', and also change their types to float

In [ ]:
df = df.withColumn('X', df['X'].cast('float')).withColumn('Y', df['Y'].cast('float'))
df = df.withColumnRenamed('X', 'Longitude').withColumnRenamed('Y', 'Latitude')

In [ ]:
# inspect the schema of the data frame
df.printSchema()

Now we can explore the dataset.
#### Count the number of incidents for each category

In [ ]:
# Method 1: SQL
df.CreateOrReplaceTempView('crime')
sqlDF = spark.sql("select Category, count(*) as Count from crime group by Category order by Count desc")
sqlDF.show(40, False)

In [ ]:
# Method 2: Data Frame
df.groupby("Category").count().orderBy("count", ascending = False).show(40, False)

In [ ]:
# Method 3: RDD
rdd = crimes.map(lambda line: (line[1], 1))
sorted(rdd.countByKey().items(), key = lambda x: -x[1])